## Imports

In [ ]:
from urllib.request import urlopen
import numpy as np
import pandas as pd
import json
import random
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import urllib
import requests
import warnings
warnings.filterwarnings("ignore")

In [ ]:
headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 92.0 Win10
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 91.0 Win10
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Firefox 90.0 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]    

## Function to get reviews data

In [ ]:
def get_review_data(url, lst):

    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers
    html = r.get(url).text
    soup = bs(html, 'html.parser')
    
    # Getting json response inside script tag
    scriptid = None
    a = soup.find_all("script", attrs={'class':None,'id':None, 'type':None})
    for x,i in zip(a, range(len(a))):
        if "userProfile" in x.string:
            scriptid = i
            
    jsonscript = a[scriptid].string[37:-141]
#     print(jsonscript)
    resp = json.loads(jsonscript)
#     print(resp)

    parent = resp.get("urqlCache")
    for x in parent:
        if (parent[x].get("data")!=None and parent[x].get("data").get('locations')!=None) and (parent[x].get("data").get('locations')[0]!=None) and (parent[x].get("data").get('locations')[0].get("reviewListPage")!=None):
            reviews = parent[x].get("data").get('locations')[0].get("reviewListPage").get("reviews")

    # Iterating through each review and getting details from the json response
    for i in range(len(reviews)):
        username = reviews[i].get("userProfile").get("displayName")
        reviewdate =  reviews[i].get("publishedDate")
        contributions = reviews[i].get("userProfile").get("contributionCounts").get("sumAllUgc")
        helpful = reviews[i].get("userProfile").get("contributionCounts").get("helpfulVote")
        revtitle = reviews[i].get("title")

        tripdetails = reviews[i].get("tripInfo")
        if tripdetails!=None:
            triptype = tripdetails.get("tripType")
            staydate = tripdetails.get("stayDate")
        else:
            triptype = None
            staydate = None

        overallrating = reviews[i].get("rating")
        reviewtext =  reviews[i].get("text")

        responsedetails = reviews[i].get("mgmtResponse")
        if responsedetails!=None:
            response = responsedetails.get("text")
            responder = responsedetails.get("username")
            responsedate = responsedetails.get("publishedDate")
            responderposition = responsedetails.get("connectionToSubject")
            
        else:
            response = None
            responder = None
            responsedate = None
            responderposition = None
            
        location = reviews[i].get("userProfile").get("hometown").get("location")
        if location!=None:
            location = location.get("additionalNames").get("long")
            
        roomtip = reviews[i].get("roomTip")
        
        ratedict = {}
        additionalratings = reviews[i].get("additionalRatings")
        if additionalratings!=None:
            for ra in additionalratings:
                ratedict[ra.get('ratingLabel')] = ra.get('rating')
            
            value = ratedict.get('Value')
            service = ratedict.get('Service')
            sleepquality = ratedict.get('Sleep Quality')
            locationrev = ratedict.get('Location')
            rooms = ratedict.get('Rooms')
            cleanliness = ratedict.get('Cleanliness')
        else:
            value = None
            service = None
            sleepquality = None
            locationrev = None
            rooms = None
            cleanliness = None
            
        lst.append([username, contributions, location, staydate, triptype, reviewdate, overallrating, revtitle, reviewtext, roomtip, value, service, sleepquality, locationrev, rooms, cleanliness, responder, responderposition, responsedate, response, helpful])
        
    return lst

## Calling the function to scrape reviews for top 10 hotels

## 1

In [74]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-"
url_part3 = "Shore_Hotel-Santa_Monica_California.html"
lst = []

# Creating a url for each review page and calling function to scrape reviews from the urls
for i in range(0, 3680, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or60-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or65-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or70-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or75-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or80-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or85-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or90-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or95-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or100-Shore_Hotel-Santa_Monica_California.html


https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or430-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or435-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or440-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or445-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or450-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or455-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or460-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or465-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or470-Shore_Hotel-Santa_Monica_Californ

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or800-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or805-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or810-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or815-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or820-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or825-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or830-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or835-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or840-Shore_Hotel-Santa_Monica_Californ

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1165-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1170-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1175-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1180-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1185-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1190-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1195-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1200-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1205-Shore_Hotel-Santa_Monica

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1530-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1535-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1540-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1545-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1550-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1555-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1560-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1565-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1570-Shore_Hotel-Santa_Monica

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1895-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1900-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1905-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1910-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1915-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1920-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1925-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1930-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or1935-Shore_Hotel-Santa_Monica

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2260-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2265-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2270-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2275-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2280-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2285-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2290-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2295-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2300-Shore_Hotel-Santa_Monica

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2625-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2630-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2635-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2640-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2645-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2650-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2655-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2660-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2665-Shore_Hotel-Santa_Monica

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2990-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or2995-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3000-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3005-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3010-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3015-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3020-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3025-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3030-Shore_Hotel-Santa_Monica

https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3355-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3360-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3365-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3370-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3375-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3380-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3385-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3390-Shore_Hotel-Santa_Monica_California.html
https://www.tripadvisor.com/Hotel_Review-g33052-d2257239-Reviews-or3395-Shore_Hotel-Santa_Monica

In [ ]:
# Converting the reviews data to a dataframe
df1 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful']) 
df1.to_csv('shorehotelreviews.csv', index=False)
# df1

## 2

In [78]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-"
url_part3 = "Hotel_Emblem_San_Francisco-San_Francisco_California.html"
lst = []
for i in range(0, 415, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df2 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df2.to_csv('hotelemblumreviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or5-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or10-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or15-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or20-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or25-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or30-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or35-Hotel_Emblem_San_Francisco-San_Francisco_California.htm

https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or325-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or330-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or335-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or340-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or345-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or350-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or355-Hotel_Emblem_San_Francisco-San_Francisco_California.html
https://www.tripadvisor.com/Hotel_Review-g60713-d112307-Reviews-or360-Hotel_Emblem_San_Francisco-San_Francisco_

## 3

In [79]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-"
url_part3 = "Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html"
lst = []
for i in range(0, 430, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df3 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df3.to_csv('hamptoninnreviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or5-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or10-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or15-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or20-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or25-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or30-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-A

https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or280-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or285-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or290-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or295-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or300-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or305-Hampton_Inn_Suites_Anaheim_Resort_Convention_Center-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d15588914-Reviews-or310-Hampton_Inn_Suites_Anaheim_Resort_Conven

## 4

In [80]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-"
url_part3 = "Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html"
lst = []
for i in range(0, 7410, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df4 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df4.to_csv('montereyplazareviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or5-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or10-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or15-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or20-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or25-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or30-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or35-

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or305-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or310-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or315-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or320-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or325-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or330-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or335-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or610-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or615-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or620-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or625-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or630-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or635-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or640-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or915-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or920-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or925-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or930-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or935-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or940-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or945-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or1220-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or1225-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or1230-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or1235-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or1240-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or1245-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or1250-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-

...

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6100-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6105-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6110-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6115-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6120-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6125-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6130-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6405-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6410-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6415-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6420-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6425-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6430-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6435-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6710-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6715-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6720-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6725-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6730-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6735-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or6740-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7015-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7020-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7025-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7030-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7035-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7040-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7045-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-

https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7320-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7325-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7330-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7335-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7340-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7345-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-d80560-Reviews-or7350-Monterey_Plaza_Hotel_Spa-Monterey_Monterey_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32737-

## 5

In [81]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-"
url_part3 = "The_Hollywood_Roosevelt-Los_Angeles_California.html"
lst = []
for i in range(0, 2840, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df5 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df5.to_csv('hollywoodrooseveltreviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or5-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or10-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or15-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or20-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or25-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or30-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or35-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g3265

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or340-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or345-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or350-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or355-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or360-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or365-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or370-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or375-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hote

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or680-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or685-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or690-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or695-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or700-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or705-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or710-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or715-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hote

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1020-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1025-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1030-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1035-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1040-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1045-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1050-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1055-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1360-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1365-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1370-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1375-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1380-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1385-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1390-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1395-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1700-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1705-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1710-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1715-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1720-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1725-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1730-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or1735-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2040-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2045-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2050-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2055-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2060-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2065-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2070-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2075-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2380-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2385-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2390-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2395-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2400-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2405-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2410-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2415-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.

https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2720-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2725-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2730-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2735-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2740-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2745-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2750-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77274-Reviews-or2755-The_Hollywood_Roosevelt-Los_Angeles_California.html
https://www.tripadvisor.

## 6

In [82]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.html"
url_part3 = "The_Anaheim_Hotel-Anaheim_California.html"
lst = []
for i in range(0, 685, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df6 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df6.to_csv('theanaheimreviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlThe_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor5-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor10-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor15-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor20-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor25-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.

https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor275-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor280-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor285-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor290-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor295-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor300-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim

https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor545-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor550-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor555-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor560-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor565-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim_California.htmlor570-The_Anaheim_Hotel-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75532-Reviews-The_Anaheim_Hotel-Anaheim

## 7

In [83]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-"
url_part3 = "Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html"
lst = []
for i in range(0, 3495, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df7 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df7.to_csv('shorecliffeviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or5-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or10-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or15-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or20-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or25-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or30-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or300-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or305-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or310-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or315-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or320-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or325-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or330-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or600-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or605-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or610-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or615-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or620-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or625-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or630-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or900-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or905-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or910-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or915-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or920-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or925-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or930-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or1200-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or1205-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or1210-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or1215-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or1220-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or1225-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or1230-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com

...

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2085-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2090-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2095-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2100-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2105-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2110-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2115-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2380-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2385-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2390-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2395-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2400-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2405-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2410-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2675-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2680-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2685-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2690-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2695-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2700-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2705-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2970-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2975-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2980-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2985-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2990-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or2995-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or3000-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com

https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or3265-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or3270-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or3275-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or3280-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or3285-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or3290-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com/Hotel_Review-g32894-d79283-Reviews-or3295-Shore_Cliff_Hotel-Pismo_Beach_San_Luis_Obispo_County_California.html
https://www.tripadvisor.com

## 8

In [84]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-"
url_part3 = "Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html"
lst = []
for i in range(0, 1980, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df8 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df8.to_csv('howardjohnsonreviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or5-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or10-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or15-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or20-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or25-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or30-Howard_Johnson_by_Wy

https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or270-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or275-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or280-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or285-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or290-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or295-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or300-Howard_

https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or540-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or545-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or550-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or555-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or560-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or565-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or570-Howard_

https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or810-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or815-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or820-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or825-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or830-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or835-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or840-Howard_

https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1080-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1085-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1090-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1095-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1100-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1105-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1110-

https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1345-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1350-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1355-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1360-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1365-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1370-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1375-

https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1610-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1615-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1620-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1625-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1630-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1635-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1640-

https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1875-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1880-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1885-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1890-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1895-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1900-Howard_Johnson_by_Wyndham_Anaheim_Hotel_and_Water_Playground-Anaheim_California.html
https://www.tripadvisor.com/Hotel_Review-g29092-d75680-Reviews-or1905-

## 9

In [85]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-"
url_part3 = "Hollywood_Hotel-Los_Angeles_California.html"
lst = []
for i in range(0, 2920, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df9 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df9.to_csv('hollywoodhotelreviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or5-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or10-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or15-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or20-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or25-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or30-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or35-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or40-Hollywood_Hotel-Los_Angeles_California.htm

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or365-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or370-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or375-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or380-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or385-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or390-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or395-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or400-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or405-Hollywood_Hotel-Los_Angeles

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or730-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or735-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or740-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or745-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or750-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or755-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or760-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or765-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or770-Hollywood_Hotel-Los_Angeles

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1095-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1100-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1105-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1110-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1115-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1120-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1125-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1130-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1135-Hollywood_Hotel-Lo

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1455-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1460-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1465-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1470-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1475-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1480-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1485-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1490-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1495-Hollywood_Hotel-Lo

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1815-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1820-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1825-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1830-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1835-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1840-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1845-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1850-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or1855-Hollywood_Hotel-Lo

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2175-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2180-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2185-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2190-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2195-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2200-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2205-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2210-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2215-Hollywood_Hotel-Lo

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2535-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2540-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2545-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2550-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2555-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2560-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2565-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2570-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2575-Hollywood_Hotel-Lo

https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2895-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2900-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2905-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2910-Hollywood_Hotel-Los_Angeles_California.html
https://www.tripadvisor.com/Hotel_Review-g32655-d77237-Reviews-or2915-Hollywood_Hotel-Los_Angeles_California.html


## 10

In [86]:
url_part1 = "https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-"
url_part3 = "Hotel_Corque-Solvang_California.html"
lst = []
for i in range(0, 1075, 5):
    if i==0:
        url_part2 = ""
    else:
        url_part2 = "or"+str(i)+"-"
#     print(url_part2)
    url = str(url_part1 + url_part2 + url_part3)
    print(url)
    lst = get_review_data(url, lst)
    
df10 = pd.DataFrame(lst, columns=['username', 'contributions', 'location', 'staydate', 'triptype', 'reviewdate', 'overallrating', 'revtitle', 'reviewtext', 'roomtip', 'value', 'service', 'sleepquality', 'locationrev', 'rooms', 'cleanliness', 'responder', 'responderposition', 'responsedate', 'response', 'helpful'])     
df10.to_csv('hotelcorquereviews.csv', index=False)

https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or5-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or10-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or15-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or20-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or25-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or30-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or35-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or40-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Review

https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or390-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or395-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or400-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or405-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or410-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or415-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or420-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or425-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or430-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g3310

https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or780-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or785-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or790-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or795-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or800-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or805-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or810-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or815-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g33103-d77933-Reviews-or820-Hotel_Corque-Solvang_California.html
https://www.tripadvisor.com/Hotel_Review-g3310